**File Name:** KariScience_withSTKWrapper.ipynb<br>
**Purpose:** Test capability to bypass STK when running Travis-CI<br>
**Author:** Emmie King (Emmie@SEE.com)<br>
**Project:** B612 - Asteroid Institute<br>
**NDA/ITAR:** None<br>
**Input:** Summary_Report.csv from KARI_ScienceOrbit 100 km -75x75.ipynb STK jupyter notebook. Size: 9x5 <br>
 - LAN (deg)  
 - Number Maneuvers  
 - Total Delta V (m/s)  
 - Mean Delta V (m/s)
 - Standard Deviation (m/s)

**Output:** None <br>
**Notes:** Travis-CI is run on a Linux platfrom. STK requires Windows and a license for the virtual environment. <br>
We are running STK and saving the output into a Summary_Report.csv file. We are testing the csv file on Windows<br>
and with Travis-CI.


In [1]:
import sys, os
from os.path import expanduser
import platform
import psutil
import pandas
import doctest
from pandas.util.testing import assert_frame_equal

In [2]:
if platform.system() == 'Windows':
    try:
        sys.path.insert(0, os.path.abspath('..')) #Works for local
        #****** Enable this block to generate Summary_Report.csv ******#
        print("Root directory path: ", os.path.abspath('..'))
        %run ./KARI_wSTK_PIDsaved.ipynb
        STK_pid = p
        processSTK = psutil.Process(STK_pid)
        processSTK.kill()

    
        #****** Enable this block to test Summary_Report.csv ******#
        #Folder and File name relative to adam_home/data/
        #STKdata_folder = '\\KARI_withSTKwrapper_2019_03_06\\Summary_Report.csv'
        #from adam import adam_config
        #ADAMpaths = adam_config.setPaths.initPaths(os.path.abspath('..'))
        #csvPath = ADAMpaths[1] + STKdata_folder
        #csvData = pandas.read_csv(csvPath)
        #print("LOCAL")
    except AttributeError:
        # In some cases EnsureDispatch('STK.Application') will return an AttributeError:
        # no attribute MinorVersion or CLSIDToClassMap. To remedy this error, 
        # delete the gen_py folder and re-run the notebook.
        import win32com
        print("Location of gen_py folder: ", win32com.__gen_path__)
else:
    sys.path.insert(0, os.path.abspath('..')) #Works for Travis CI
    #print("Root directory path: ", os.path.abspath('.'))
    from adam import adam_config
    
    #Folder and File name relative to adam_home/
    STKdata_folder = '/KARI/2019_04_01/Summary_Report 100 km-75x75.csv'
    ADAMpaths = adam_config.setPaths.initPaths(os.path.abspath('..')) #adam_home path
    csvPath = ADAMpaths[1] + STKdata_folder
    csvData = pandas.read_csv(csvPath)
    #print("REMOTE")    

C:\Users\macuser\AppData\Local\Temp\gen_py\3.7
Root directory path:  C:\Users\macuser\TestingTravisCI\adam_home


**Method Testing** The following cells should only be run if you are testing the previous cells for errors.<br>
The CSV file being tested here was generated on Windows Platform. We are bypassing TravisCI<br>
and checking that the numbers match the file uploaded to the Repository.<br>
If the numbers match, we will allow TravisCI to pass the notbeook.<br>

In [77]:
def csvExist(csvPath):
    '''
    This will test if the file exists.
    >>> assert csvExist(csvPath)==True
    '''

    return os.path.exists(csvPath) == 1

In [78]:
def csvEmpty(csvPath):
    '''
    This will test if the file is empty and tests the size of the file in bytes.
    >>> assert csvEmpty(csvPath)==False
    '''
    return os.stat(csvPath).st_size==0

In [89]:
def csvShape(csvData):
    '''
    This will test the number of rows and columns in the file.
    >>> assert csvShape(csvData)==(9, 6)
    '''
    return csvData.shape

In [79]:
def csvLAN(csvData):
    '''
    This will test if the values in LAN (deg) is expected.
    >>> assert csvLAN(csvData)==0.0
    '''
    return csvData.loc[0,'LAN (deg)']

In [80]:
def csvNMan(csvData):
    '''
    This will test if the values in the file are as expected.
    >>> assert csvNMan(csvData)==12
    '''
    return csvData.loc[0,'Number Maneuvers']

In [90]:
# add this line at the end of the notebook
doctest.testmod(verbose=True)

Trying:
    assert csvEmpty(csvPath)==False
Expecting nothing
ok
Trying:
    assert csvExist(csvPath)==True
Expecting nothing
ok
Trying:
    assert csvLAN(csvData)==0.0
Expecting nothing
ok
Trying:
    assert csvMeanDelV(csvData)==4.927335933687437
Expecting nothing
**********************************************************************
File "__main__", line 4, in __main__.csvMeanDelV
Failed example:
    assert csvMeanDelV(csvData)==4.927335933687437
Exception raised:
    Traceback (most recent call last):
      File "C:\Users\macuser\Anaconda3\lib\doctest.py", line 1329, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__.csvMeanDelV[0]>", line 1, in <module>
        assert csvMeanDelV(csvData)==4.927335933687437
    AssertionError
Trying:
    assert csvNMan(csvData)==12
Expecting nothing
ok
Trying:
    assert csvShape(csvData)==(9, 6)
Expecting nothing
ok
Trying:
    assert csvStdDev(csvData)==1.0559566325090377
Expecting nothing
******************************

TestResults(failed=3, attempted=9)

In [23]:
#Restore saved information after kernel restart and cell output are cleared
#%store -r summaryReportDF
#print(summaryReportDF)